In [ ]:
# need 100GB session to run this code (or maybe way less; like just 10GB; idk)
# Using this to make EU-ASPECT files for Yeager. For E3SM SMYLE. Does SE->FV regridding
# Can be used for variety of variables, dims, forecast lengths, etc. And it's a shorty.
# in progress. July 28, 2025 omg.

import xarray as xr
import numpy as np
import glob
import os
from IPython.display import clear_output
from geocat.comp import interp_hybrid_to_pressure

month = "05"
varname = "U"
varname_out = "U"
year_start = 1981
year_end = 2021
srcDir = "/glade/campaign/cgd/ccr/E3SMv2.1-SMYLEsmbb/archive"
destDir = "/glade/derecho/scratch/sglanvil/holdingCell/SMYLE_E3SM"
tmpDir = "/glade/derecho/scratch/sglanvil/holdingCell"
weightFile = "/glade/work/strandwg/E3SM_regridding/maps/map_ne30pg2_to_0.9x1.25_aave.20230801.nc"

data = []
times=[]
for year in range(year_start, year_end + 1):
    clear_output(wait=True)  # Clears the current output
    print(year)
    pattern = f"{srcDir}/M{month}/v21.LR.BSMYLEsmbb.{year}-{month}.*/atm/proc/tseries/month_1/*.eam.h0.{varname}.*.nc"
    memb_files = sorted(glob.glob(pattern))
    memb_data = []
    for inFile in memb_files:
        mappedFile = f"{tmpDir}/tmp_{os.path.basename(inFile)}"
        cmd = f'module load nco && ncremap -m {weightFile} -in {inFile} {mappedFile}'  # ----- Regrid E3SM
        os.system(f'bash -lc "{cmd} > /dev/null 2>&1"')
        ds = xr.open_dataset(mappedFile,decode_times=False)
        os.remove(mappedFile)
        da = ds[varname].expand_dims(M=[0])
        memb_data.append(da)
    year_data = xr.concat(memb_data, dim="M").expand_dims(Y=[year])
    data.append(year_data)
    times.append(xr.open_dataset(memb_files[0])["time"].drop_vars("time", errors="ignore").expand_dims("Y"))
data = xr.concat(data, dim = "Y").rename({"time": "L"})
times = xr.concat(times, dim="Y").rename({"time": "L"})
hyam = ds["hyam"]
hybm = ds["hybm"]
ds = data.to_dataset(name=varname_out)
ds["time"] = times
if varname in ["U", "V"]:
    ds["hyam"] = hyam
    ds["hybm"] = hybm
ds = ds.assign_coords(M=np.arange(1, ds.sizes["M"] + 1))
ds = ds.assign_coords(L=np.arange(1, ds.sizes["L"] + 1))
ds.to_netcdf(f"{destDir}/{varname_out}_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")

1984


### Deal with total precip calculation (PRECC+PRECL)

In [ ]:
# Deal with total precip calculation (PRECC+PRECL)
ds_precc = xr.open_dataset(f"{destDir}/PRECC_1981-2021_M05_E3SM_SMYLE.nc")
ds_precl = xr.open_dataset(f"{destDir}/PRECL_1981-2021_M05_E3SM_SMYLE.nc")
pr = ds_precc["PRECC"] + ds_precl["PRECL"]
pr.attrs = ds_precc["PRECC"].attrs
pr.attrs["long_name"] = "Total precipitation (convective + large-scale)"
ds = pr.to_dataset(name="pr")
ds.to_netcdf(f"{destDir}/pr_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")

### Deal with ua10, ua250, ua850, va250, va850

In [4]:
# Deal with ua10, ua250, ua850, va250, va850
varname_out = "ua850"
target = 85000.
ds_U = xr.open_dataset(f"{destDir}/U_1981-2021_M05_E3SM_SMYLE.nc")
ds_PS = xr.open_dataset(f"{destDir}/PS_1981-2021_M05_E3SM_SMYLE.nc")
data = interp_hybrid_to_pressure(ds_U["U"], ds_U['hyam'], ds_U['hybm'], ds_PS["PS"], 
                                 p0=1e5, new_levels = np.array([target]), 
                                 method = 'linear').squeeze()
ds = data.to_dataset(name=varname_out)
ds.to_netcdf(f"{destDir}/{varname_out}_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")